# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
%%capture
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import re
import numpy as np
# nltk
import nltk
nltk.download('stopwords')
nltk.download('wordnet') # download for lemmatization
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
# sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
# from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
# other models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# pickle
import pickle

[nltk_data] Downloading package stopwords to /home/martin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/martin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/martin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# load data from database
engine = create_engine('sqlite:///DisasterData.db')
df = pd.read_sql_table('TextMessages', engine)
X = df[["message", "original", "genre"]]
Y = df.drop(columns= ["id", "message", "original", "genre"])

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    # Normalization
    
    # Convert to lower case
    text = text.lower()
    
    # Remove punctuation characters - this regex finds everything which is not a combination of letters
    # and numbers and replaces it with a whitespace
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    
    # Tokenization
    
    # Split into tokens
    words = word_tokenize(text)
    
    
    # Remove stopwords
    words = [w for w in words if w not in stopwords.words("english")]
    
    # Part-of-speech tagging maybe useful here?
    # Named Entity Recognition usefuk here?
    
    # Stemming - only keep the stem of a word, simple find and replace method which removes f.e. "ing"
    # stemmed = [PorterStemmer().stem(w) for w in words]
    
    # Lemmatization - more complex appraoch using dictionaries which can f.e. map "is" and "was" to "be"
    # Lemmatize verbs by specifying pos
    lemmed_verbs = [WordNetLemmatizer().lemmatize(w, pos='v') for w in words]
    # Reduce nouns to their root form
    lemmed_nouns = [WordNetLemmatizer().lemmatize(w) for w in lemmed_verbs]
    return lemmed_nouns

In [6]:
# Split the data in training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size = 0.05) # We drastically decrease the train_size to allow our GridSearch to run in a feasible amount of time

In [15]:
# Calculate the average accuracy for each target column
def print_acc(name, model, y_test, y_pred):
    columns = y_test.columns
    y_pred_df = pd.DataFrame(y_pred, columns = columns)
    accuracy = (y_pred_df == y_test.reset_index().drop(["index"], axis = 1)).mean()
    report = classification_report(y_true = y_test,
                              y_pred = y_pred,
                              target_names = list(y_test.columns),
                            #  output_dict = True,
                              zero_division = 0)
    print(f"F1 score, recall and precision per category {name}: \n")
    # print(f"Average accuracy: {accuracy.mean()}")
    # print(accuracy)
    print(report)
    
    return {'name' : name, 'model': model, 'report' : report}

In [8]:
# Create an empty array to store all the results and the models to find the best one in the end
results = []

# Native model without optimization (MultiOutputClassifier with RandomForestClassifier)

In [9]:
# pipeline = Pipeline([
#         ('features', FeatureUnion([

#             ('text_pipeline', Pipeline([
#                 ('vect', CountVectorizer(tokenizer=tokenize)),
#                 ('tfidf', TfidfTransformer())
#             ]))
#         ])),

#         ('clf', MultiOutputClassifier(RandomForestClassifier()))
#     ])

random_forest_pipe = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

random_forest_pipe.fit(X_train["message"], y_train)
y_pred = random_forest_pipe.predict(X_test["message"])

In [10]:
results.append(print_acc("MultiOutputClassifier RandomForest", random_forest_pipe, y_test, y_pred))

F1 score, recall and precision per category MultiOutputClassifier RandomForest: 

{'related': {'precision': 0.8064771627211124, 'recall': 0.9589284779992675, 'f1-score': 0.8761203661655393, 'support': 19113}, 'request': {'precision': 0.7895659798334064, 'recall': 0.42486435480066054, 'f1-score': 0.5524539877300614, 'support': 4239}, 'offer': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 112}, 'aid_related': {'precision': 0.7265138154027043, 'recall': 0.598624297616741, 'f1-score': 0.6563977266691454, 'support': 10322}, 'medical_help': {'precision': 0.75, 'recall': 0.0015113350125944584, 'f1-score': 0.0030165912518853692, 'support': 1985}, 'medical_products': {'precision': 0.8, 'recall': 0.003218020917135961, 'f1-score': 0.00641025641025641, 'support': 1243}, 'search_and_rescue': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 697}, 'security': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 454}, 'military': {'precision': 0.0, 'recall': 0.0, 

# kNN

In [9]:
# knn_pipe = Pipeline([
#         ('vect', CountVectorizer(tokenizer=tokenize)),
#         ('tfidf', TfidfTransformer()),
#         ('clf', KNeighborsClassifier())
#     ])
# knn_pipe.fit(X_train["message"], y_train)
# y_pred_knn = knn_pipe.predict(X_test["message"])

In [10]:
# results.append(print_acc("kNN", knn_pipe, y_test, y_pred_knn))

# Decision tree


In [11]:
# decision_tree_pipe = Pipeline([
#         ('vect', CountVectorizer(tokenizer=tokenize)),
#         ('tfidf', TfidfTransformer()),
#         ('clf', DecisionTreeClassifier())
#     ])
# decision_tree_pipe.fit(X_train["message"], y_train)
# y_pred_decision_tree = decision_tree_pipe.predict(X_test["message"])

In [12]:
# results.append(print_acc("Decision Tree", decision_tree_pipe, y_test, y_pred_decision_tree))

# Random Forest

In [13]:
# random_forest_only_pipe = Pipeline([
#         ('vect', CountVectorizer(tokenizer=tokenize)),
#         ('tfidf', TfidfTransformer()),
#         ('clf', RandomForestClassifier())
#     ])
# random_forest_only_pipe.fit(X_train["message"], y_train)
# y_pred_random_forest_only = random_forest_only_pipe.predict(X_test["message"])

In [14]:
# results.append(print_acc("Random Forest", random_forest_only_pipe, y_test, y_pred_random_forest_only))

In [15]:
# for result in results:
#   print(result["name"])
#   print(result["accuracy"].mean())

# Improve models using GridSearch

## MultiOutputClassifier + RandomForestClassifier

In [16]:
# Check for available parameters to optimize
# random_forest_pipe.get_params().keys()

In [17]:
# parameters_mo_rf = {
#     # vect
#     # https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

    
#     # tfidf
#     # https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html
#     'tfidf__norm' : ['l1', 'l2'],
#   #  'tfidf__use_idf' : [True, False],
#    # 'tfidf__smooth_idf': [True, False],
#    # 'tfidf__sublinear_tf' : [True, False],

#     # clf
#     # https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
#     'clf__estimator__criterion' : ['gini', 'entropy'],
#     'clf__estimator__n_estimators': [50, 100, 150, 200],
#     'clf__estimator__max_depth' : [None, 5, 10],
# }

# cv_parameters_mo_rf = GridSearchCV(random_forest_pipe, param_grid=parameters_mo_rf) 
# cv_parameters_mo_rf.fit(X_train["message"], y_train)
# y_pred_mo_rf_cv = cv_parameters_mo_rf.predict(X_test["message"])

In [18]:
# results.append(print_acc("MultiOutputClassifier Random Forest CV", cv_parameters_mo_rf, y_test, y_pred_mo_rf_cv))

## kNN

In [20]:
# knn_pipe.get_params().keys()

In [21]:
# parameters_knn = {
#     # vect
#     # https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

    
#     # tfidf
#     # https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html
#     'tfidf__norm' : ['l1', 'l2'],
#   #  'tfidf__use_idf' : [True, False],
#   #  'tfidf__smooth_idf': [True, False],
#   #  'tfidf__sublinear_tf' : [True, False],

#     # clf
#     # https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
#     'clf__n_neighbors' : [3, 5, 8],
#     'clf__weights' : ['uniform', 'distance'],
#     'clf__algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],

# }

# cv_knn = GridSearchCV(knn_pipe, param_grid=parameters_knn) 
# cv_knn.fit(X_train["message"], y_train)
# y_pred_knn_cv = cv_knn.predict(X_test["message"])

In [22]:
# results.append(print_acc("kNN CV", cv_knn, y_test, y_pred_mo_rf_cv))

# Classification report

In [23]:
from sklearn.metrics import classification_report

In [38]:
report = classification_report(y_true = y_test,
                              y_pred = y_pred,
                              target_names = list(y_test.columns),
                              output_dict = True,
                              zero_division = 0)

# Evaluate the results

In [25]:
for result in results:
  print(result["name"])
  print(result["accuracy"].mean())

MultiOutputClassifier RandomForest
0.9399697146986956
kNN
0.9303656032396095
Decision Tree
0.9243796675499878
Random Forest
0.9397701070310077
MultiOutputClassifier Random Forest CV
0.9394546351424211
kNN CV
0.9394546351424211


As we can see, the models performed all very similar. Only the decision tree model is a bit worse compared to the other ones. Surprisingly, our unoptimized orginal model with a MultiOutpuClassfier and a RandomForestClassifier performed best. Therefore we can assume that the standard model configuration fits good to our problem and the optimization attempt only leads us away from the optimum. 94% is a quite good result so we can stick with that model.

In [30]:
best_model = results[0]['model']

Now that we found the best model configuration, we retrain the model with 80% of the data

In [34]:
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X, Y, train_size = 0.80)
best_model.fit(X_train_new["message"], y_train_new)
y_pred_final = best_model.predict(X_test_new["message"])

### 9. Export your model as a pickle file

In [37]:
model_params = best_model.get_params()
model = best_model

fileObj = open('model_params.obj', 'wb')
pickle.dump(model_params,fileObj)
fileObj.close()

fileObj = open('model.obj', 'wb')
pickle.dump(model,fileObj)
fileObj.close()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.